In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import emcee

In [ ]:
def standdev(array):
    array = np.array(array)
    mean = np.mean(array)
    minarr = (array - mean)
    minsarrs = minarr**2
    meanarray = np.sum(minsarrs) / (len(array) - 1)
    std = np.sqrt(meanarray)
    print('Mean:', mean, 'RMS:', std)
    return mean, std


def LumCompare(new, trusted, rad):
    steven = Table.read(new)
    mcxc = Table.read(trusted)
    names = steven['NAME']
    RA = steven["RA_detc"]
    DEC = steven["DEC_detc"]
    ZS = steven['REDSHIFT']
    ZM = mcxc['z']
    LumS = steven['LUM_CORR']   # units of ergs/s
    # applying the conversion from the fits file to it, dont *1e+37 again
    LumM = mcxc['L500'] * 1e+37

    #Establishing the coordinates of the two tables
    c1 = SkyCoord(steven["RA_detc"] * u.deg, steven["DEC_detc"] * u.deg)
    c2 = SkyCoord(mcxc["RAdeg"], mcxc["DEdeg"])
    #Finding the 5 arcminute matches
    #Guide: idxN_a indicies of NVSS sources that match to corresponding element
    # of idxP1_a which contains the indicies of psz1 sources
    #idxP1_a indicies of psz1 sources that match to the corresponding element
    # of idxN_a which contains the indicies of nvss sources
    #n = [1, 2, 3]
    #p = [23, 5, 23]
    #The radio source of index 1 and 3 both match the psz source of index 23

    idmcxc, idsteven, d2d, d3d = c1.search_around_sky(c2, rad * u.arcmin)
    print('MCXC Entries', len(idmcxc), 'Steven Entries', len(idsteven))

    #checking if there are any empty elements/ there aren't this only
    #flags the 0 indicies
    a = [1 for idp in idsteven if not idp]
    print('Empty Entries in Steven', sum(a))
    b = [1 for idp in idmcxc if not idp]
    print('Empty Entries in MCX', sum(b))
    stevenunique = np.unique(idsteven)
    print('Unique Steven', len(stevenunique))
    mcxcunique = np.unique(idmcxc)
    print('Unique MCXC', len(mcxcunique))
    #Filtering variables based on position matches
    names = names[idsteven]
    RA = RA[idsteven]
    DEC = DEC[idsteven]
    LumSteven = LumS[idsteven]
    LumMCXC = LumM[idmcxc] * 1e7  # applying luminosity unit conversion
    ZSteven = ZS[idsteven]
    ZMCXC = ZM[idmcxc]

    return ZSteven, LumSteven, ZMCXC, LumMCXC, names, RA, DEC


def findoutliers(name, newz, trustedz, diffguess, ra, dec):
    name = np.asarray(name)
    newz = np.asarray(newz)
    trustedz = np.asarray(trustedz)
    ra = np.asarray(ra)
    dec = np.asarray(dec)
    zdiff = newz - trustedz
    offone = np.flatnonzero(np.abs(zdiff) >= np.abs(diffguess))
    print('Redshift Difference:')
    print('Name:  ', 'Steven_Z', 'MCXC_Z', 'Z_Diff', 'RA', 'DEC', sep='      ')
    for i in offone:
        print(name[i],
              newz[i],
              trustedz[i],
              zdiff[i],
              ra[i],
              dec[i],
              sep='   ')
    mean, std = standdev(zdiff)
    return mean, std, offone


def LumCompare2(new, trusted, rad):
    steven = Table.read(new)
    mcxc = Table.read(trusted)
    expos = steven["EXPTIME"]
    index = steven['INDEX']
    dup = steven['DUPLICATE']
    L1 = mcxc['L500r1']
    L2 = mcxc['L500r2']
    L3 = mcxc['L500r3']
    L4 = mcxc['L500r4']
    oname = mcxc['OName']
    aname = mcxc['AName']

    LumEup = steven['LUM_CORR_POSERR']  # units of ergs/s
    LumEdown = steven['LUM_CORR_NEGERR']  # units of ergs/s
    Corr = steven['CORR']

    #Establishing the coordinates of the two tables
    c1 = SkyCoord(steven["RA_detc"] * u.deg, steven["DEC_detc"] * u.deg)
    c2 = SkyCoord(mcxc["RAdeg"], mcxc["DEdeg"])
    #Finding the 5 arcminute matches
    #Guide: idxN_a indicies of NVSS sources that match to corresponding element
    # of idxP1_a which contains the indicies of psz1 sources
    #idxP1_a indicies of psz1 sources that match to the corresponding element
    # of idxN_a which contains the indicies of nvss sources
    #n = [1, 2, 3]
    #p = [23, 5, 23]
    #The radio source of index 1 and 3 both match the psz source of index 23

    idmcxc, idsteven, d2d, d3d = c1.search_around_sky(c2, rad * u.arcmin)
    print('MCXC Entries', len(idmcxc), 'Steven Entries', len(idsteven))

    #checking if there are any empty elements/ there aren't this only
    #flags the 0 indicies
    a = [1 for idp in idsteven if not idp]
    print('Empty Entries in Steven', sum(a))
    b = [1 for idp in idmcxc if not idp]
    print('Empty Entries in MCX', sum(b))
    stevenunique = np.unique(idsteven)
    print('Unique Steven', len(stevenunique))
    mcxcunique = np.unique(idmcxc)
    print('Unique MCXC', len(mcxcunique))

    L1 = L1[idmcxc]
    L2 = L2[idmcxc]
    L3 = L3[idmcxc]
    L4 = L4[idmcxc]
    oname = oname[idmcxc]
    aname = aname[idmcxc]
    expos = expos[idsteven]
    dup = dup[idsteven]
    index = index[idsteven]
    LumEP = LumEup[idsteven]
    LumEN = LumEdown[idsteven]
    Corr = Corr[idsteven]
    return LumEP, LumEN, expos, index, L1, Corr, oname, aname, dup


def Chi2RatioError(data, err):
    model = np.ones(len(data))
    chisq = np.sum((data - model)**2 / (err)**2)
    print('Chi Square Statistic:(Error)', chisq)
    return chisq


def log_prior(theta):
    A = theta
    if not -10 < A < 10:
        return -np.inf
    return 0


def log_likelihood(theta, LumMCXC, data, err):
    A = theta
    model = A * (LumMCXC**1.0)
    lnchisq = np.sum(-0.5 * (data - model)**2 / err**2)
    return lnchisq


def log_posterior(theta, LumMCXC, data, err):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return log_prior(theta) + log_likelihood(theta, LumMCXC, data, err)


def MCMCfitandPlot(data, error, LumMCXC):
    ndim = 1  # number of parameters in the model
    nwalkers = 100  # number of MCMC walkers
    nburn = 100  # "burn-in" period to let chains stabilize
    nsteps = 500  # number of MCMC steps to take

    pos = [
        np.array([1.01]) + 5e-2 * np.random.randn(ndim)
        for i in range(nwalkers)
    ]

    sampler = emcee.EnsembleSampler(nwalkers,
                                    ndim,
                                    log_posterior,
                                    args=[LumMCXC, data, error])
    mcmcgunk = sampler.run_mcmc(pos, nsteps)
#     one = sampler.chain[0, 9999, 0]
#     print('A:', sampler.chain[0, 9999, 0])
#     mcmcnumbers = (one)  #(one, two, three)

    plt.figure(1)
    for j in range(ndim):
        for i in range(nwalkers):
            derp = sampler.chain[i, :, j]
            plt.plot(derp, linewidth='0.25')
        plt.show()

    label = ('A')
    samples = sampler.chain[:, nburn:, :].reshape((-1, ndim))
    #     fig = corner.corner(samples, labels=label)
    #     plt.show()
    thechain = sampler.flatchain
    print('Mean values:', np.mean(thechain, axis=0))
    print('Standard deviation:', np.std(thechain, axis=0))
    return sampler


def IntScat(param, data, model, error):
    #param number of parameters that was fitted for
    a = r'\sigma^2'
    N = len(data)
    #mean = np.mean(data)
    minus = (data - model) / model
    minus2 = minus**2
    var = np.sum(minus2) / (N - param)
    errvar = np.sum((error / model)**2) / N
    intscat = var - errvar
    print(r"The instrinsic scatter sigma^2 = ", intscat)
    print(r"The instrinsic scatter sigma = ", np.sqrt(intscat))
    return intscat

In [ ]:
ZSteven, LumSteven, ZMCXC, LumMCXC, names, RA, DEC = LumCompare(
    './catalogs/results_table_inspected_0.1_2.4.fits', './catalogs/MCXC.fits',
    1)
LumEP, LumEN, expos, index, L1, Corr, Oname, Aname, Dup = LumCompare2(
    './catalogs/results_table_inspected_0.1_2.4.fits', './catalogs/MCXC.fits',
    1)

In [ ]:
lumfilt = np.flatnonzero(LumSteven > 0.0)
len(lumfilt)

In [ ]:
#Applying luminosity filter
ZSteven, LumSteven, ZMCXC, LumMCXC, names, RA, DEC, LumEP, LumEN, expos, index, L1, Corr, Oname, Aname, Dup = ZSteven[
    lumfilt], LumSteven[lumfilt], ZMCXC[lumfilt], LumMCXC[lumfilt], names[
        lumfilt], RA[lumfilt], DEC[lumfilt], LumEP[lumfilt], LumEN[
            lumfilt], expos[lumfilt], index[lumfilt], L1[lumfilt], Corr[
                lumfilt], Oname[lumfilt], Aname[lumfilt], Dup[lumfilt]

In [ ]:
dupfilt = np.where(Dup == 'False')
len(Dup)

In [ ]:
expfilt = np.flatnonzero(expos >= 1000.0)
len(expfilt)

In [ ]:
#Applying exposure filter
ZStevenef, LumStevenef, ZMCXCef, LumMCXCef, namesef, RAef, DECef, LumEPef, LumENef, exposef, indexef, L1ef, Corref, Onameef, Anameef = ZSteven[
    expfilt], LumSteven[expfilt], ZMCXC[expfilt], LumMCXC[expfilt], names[
        expfilt], RA[expfilt], DEC[expfilt], LumEP[expfilt], LumEN[
            expfilt], expos[expfilt], index[expfilt], L1[expfilt], Corr[
                expfilt], Oname[expfilt], Aname[expfilt]

In [ ]:
zfilt = np.flatnonzero(ZStevenef >= 0.1)
len(zfilt)

In [ ]:
#Applying redshift filter
ZStevenefzf, LumStevenefzf, ZMCXCefzf, LumMCXCefzf, namesefzf, RAefzf, DECefzf, LumEPefzf, LumENefzf, exposefzf, indexefzf, L1efzf, Correfzf, Onameefzf, Anameefzf = ZStevenef[
    zfilt], LumStevenef[zfilt], ZMCXCef[zfilt], LumMCXCef[zfilt], namesef[
        zfilt], RAef[zfilt], DECef[zfilt], LumEPef[zfilt], LumENef[
            zfilt], exposef[zfilt], indexef[zfilt], L1ef[zfilt], Corref[
                zfilt], Onameef[zfilt], Anameef[zfilt]

In [ ]:
lowLfilt = np.flatnonzero(LumMCXCefzf >= 1e44)
len(lowLfilt)

In [ ]:
ZStevenefzf, LumStevenefzf, ZMCXCefzf, LumMCXCefzf, namesefzf, RAefzf, DECefzf, LumEPefzf, LumENefzf, exposefzf, indexefzf, L1efzf, Correfzf, Onameefzf, Anameefzf = ZStevenefzf[
    lowLfilt], LumStevenefzf[lowLfilt], ZMCXCefzf[lowLfilt], LumMCXCefzf[
        lowLfilt], namesefzf[lowLfilt], RAefzf[lowLfilt], DECefzf[
            lowLfilt], LumEPefzf[lowLfilt], LumENefzf[lowLfilt], exposefzf[
                lowLfilt], indexefzf[lowLfilt], L1efzf[lowLfilt], Correfzf[
                    lowLfilt], Onameefzf[lowLfilt], Anameefzf[lowLfilt]

Steven as I mentioned last time, I'm not sure if simply taking the outliers out is something that we want to do, but that's how my code is currently set up. It takes the redshift outliers out and the most extreme luminosity outlier from the low end. 
If you want to include the outliers just remove the steps that do that.

## Find Redshift Outliers, Track, and Remove

In [ ]:
mean, std, outind = findoutliers(namesefzf, ZStevenefzf, ZMCXCefzf, 0.0009,
                                 RAefzf, DECefzf)
Routind = outind[::-1]
len(Routind)

In [ ]:
#Remove Outliers

print(len(namesefzf))
zsteven = np.delete(ZStevenefzf, outind)
Lsteven = np.delete(LumStevenefzf, outind)
zmcxc = np.delete(ZMCXCefzf, outind)
Lmcxc = np.delete(LumMCXCefzf, outind)
name = np.delete(namesefzf, outind)
ra = np.delete(RAefzf, outind)
dec = np.delete(DECefzf, outind)
LstevenEP = np.delete(LumEPefzf, outind)
LstevenEN = np.delete(LumENefzf, outind)
exptime = np.delete(exposefzf, outind)
idxs = np.delete(indexefzf, outind)
L1mcxc = np.delete(L1efzf, outind)
corr = np.delete(Correfzf, outind)
oname = np.delete(Onameefzf, outind)
aname = np.delete(Anameefzf, outind)
print(len(name))

### In case you want to keep the outliers. Comment out the above and uncomment the code below.

#zsteven = ZStevenefzf
#Lsteven = LumStevenefzf
#zmcxc = ZMCXCefzf
#Lmcxc = LumMCXCefzf,
#name = namesefzf
#ra = RAefzf
#dec = DECefzf,
#LstevenEP = LumEPefzf
#LstevenEN = LumENefzf
#exptime = exposefzf
#idxs = indexefzf
#L1mcxc = L1efzf
#corr = Correfzf,
#oname = Onameefzf,
#aname = Anameefzf,

In [ ]:
#Track Outliers
###########################################################
zsteveno = ZStevenefzf[outind]
Lsteveno = LumStevenefzf[outind]
zmcxco = ZMCXCefzf[outind]
Lmcxco = LumMCXCefzf[outind]
nameo = namesefzf[outind]
rao = RAefzf[outind]
deco = DECefzf[outind]
LstevenEPo = LumEPefzf[outind]
LstevenENo = LumENefzf[outind]
exptimeo = exposefzf[outind]
idxso = indexefzf[outind]
L1mcxco = L1efzf[outind]
corro = Correfzf[outind]
onameo = Onameefzf[outind]
anameo = Anameefzf[outind]
##############################################################

## Luminosity Ratios


In [ ]:
#Getting Rid of Big Outlier
outthree = np.flatnonzero(Lsteven / Lmcxc == np.min(Lsteven / Lmcxc))
print('index', outthree, 'length of array with index', len(Lmcxc))
print(name[outthree], idxs[outthree], ra[outthree], dec[outthree],
      zsteven[outthree], Lsteven[outthree], LstevenEP[outthree],
      LstevenEN[outthree], Lmcxc[outthree], exptime[outthree], corr[outthree])

In [ ]:
zsteven = np.delete(zsteven, outthree)
Lsteven = np.delete(Lsteven, outthree)
zmcxc = np.delete(zmcxc, outthree)
Lmcxc = np.delete(Lmcxc, outthree)
name = np.delete(name, outthree)
ra = np.delete(ra, outthree)
dec = np.delete(dec, outthree)
LstevenEP = np.delete(LstevenEP, outthree)
LstevenEN = np.delete(LstevenEN, outthree)
exptime = np.delete(exptime, outthree)
idxs = np.delete(idxs, outthree)
L1mcxc = np.delete(L1mcxc, outthree)
corr = np.delete(corr, outthree)
oname = np.delete(oname, outthree)
aname = np.delete(aname, outthree)


In [ ]:
#Big Outlier
print(6.13709651e+43 / 3.725047e+44)

In [ ]:
Ratio = Lsteven / Lmcxc
meanlum, stdlum = standdev(Ratio)
Iolow, Iomid, Iohigh = np.percentile(Ratio, [16, 50, 84.0])
print(Iomid, '+', Iohigh - Iomid, '-', Iomid - Iolow)

In [ ]:
plt.figure(figsize=(7, 7 * (np.sqrt(5.) - 1.0) / 2.0))
plt.hist(Ratio, bins=40, alpha=0.5, label='Lum 0.1-2.4keV')
plt.axvline(x=Iomid, c='#A60628', label='Median')
plt.axvline(x=Iolow, c='#A60628', linestyle='--')
plt.axvline(x=Iohigh, c='#A60628', linestyle='--')
plt.axvline(x=meanlum, color='black', label='Mean')
plt.axvline(x=meanlum - stdlum, color='black', linestyle='--')
plt.axvline(x=meanlum + stdlum, color='black', linestyle='--')
plt.xlabel('Luminosity Ratio')
plt.ylabel('Number')
#plt.title(r'Luminosity Ratio $L_{\rm{Corr}}/L_{\rm{MCXC}}$')
plt.legend(loc='upper right', frameon=False)
plt.show()


In [ ]:
plt.tight_layout()
plt.savefig('./figures/LumRatioHist.pdf', bbox='tight')

## Luminosity Fit

### Finding Single Error for the Fit

In [ ]:
#Taking the larger value
singleerr = []
for (a, b) in zip(LstevenEN, LstevenEP):
    if (np.abs(a) > np.abs(b)):
        singleerr.append(a)
    elif (np.abs(b) > np.abs(a)):
        singleerr.append(b)
    else:
        singleerr.append(a)

#The positive and the negative errors are the same.
singleerr = np.array(singleerr)

## Single Parameter Fit    $Y = Ax$

In [ ]:
sampler = MCMCfitandPlot(Lsteven / 1e44, singleerr / 1e44, Lmcxc / 1e44)

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)

In [ ]:
yfit  = flat_samples[:, None] * (Lmcxc / 1e44) ** 1.0
mu = yfit.mean(0)
sig = yfit.std(0)

In [ ]:
A1s = flat_samples.T[0].mean()
y1s = A1s * ((Lmcxc / 1e44)**1.0)

res1 = (y1s - (Lsteven / 1e44)) / (singleerr / 1e44)
resone = ((Lsteven / 1e44) - y1s) / (y1s)
print('double check length of array', len(resone))

In [ ]:
xLCs = Lmcxc / 1e44
yLCs = xLCs

f = plt.figure(figsize=(7 * (np.sqrt(5.) - 1.0) / 2.0, 7))
ax = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
axs = plt.subplot2grid((3, 1), (2, 0))

#MCXC on x axis becuase it is orderd already
ax.errorbar(Lmcxc / 1e44,
             Lsteven / 1e44,
             yerr=singleerr / 1e44,
             fmt='.',
             c='#348ABD',
             label="This Work")
ax.plot([0.1,100], [0.1,100], color='k', lw=1, label='1:1')
ax.plot(Lmcxc / 1e44, y1s, color='#A60628', label="Fit")

ax.tick_params(labelbottom=False)    #Remove x-tic labels for the first frame

ax.set_xlim(1, 25)
ax.set_ylim(1, 25)
ax.set_ylabel('L$_x$ [$10^{44}~erg~s^{-1}$]')
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend()


axs.plot(Lmcxc / 1e44, resone, '.', c='#348ABD')
axs.axhline(y=0.0, color='k', zorder=0)

axs.set_ylabel('($L_x$ - $L_{fit}$) / $L_{fit}$')
axs.set_xlabel('L$_{MCXC}$ [$10^{44}~erg~s^{-1}$]')
axs.set_xlim(1, 25)
axs.set_xscale('log')


In [ ]:
plt.tight_layout()
plt.savefig('./figures/LumFit_SingleParam.pdf', bbox='tight')

In [ ]:
chisqresult2 = np.sum( (Lsteven/1e44 - y1s) ** 2 / (singleerr/1e44) ** 2)
chisqresult2

In [ ]:
intscat1s = IntScat(1,Lsteven/1e44,y1s,singleerr/1e44)